In [37]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
#Loading Files
#Loading all the jobs with or without salary contained in the following file 
df_ws = pd.read_csv('F:/Data_Cleaned_NoSalary.csv', index_col=0)
#Loading all the jobs having salary contained in the following file 
df_os = pd.read_csv('F:/Data_Cleaned_OnlySalary.csv', index_col=0)



In [39]:
#karachi
df_os['karachi_yn'] = df_os['Location'].apply(lambda x: 1 if 'karachi' == x.lower() else 0)
#lahore
df_os['lahore_yn'] = df_os['Location'].apply(lambda x: 1 if 'lahore' == x.lower() else 0)
#islamabad
df_os['isl_yn'] = df_os['Location'].apply(lambda x: 1 if 'islamabad' == x.lower() else 0)
#other_loc
df_os['other_loc'] = df_os['Location'].apply(lambda x: 1 if 'islamabad' != x.lower() and 'karachi' != x.lower() and 'lahore' != x.lower()  else 0)
df_os['other_loc'].value_counts()

0    37
1     5
Name: other_loc, dtype: int64

<h1> Decision Tress </h1>

In [40]:
df_os.columns

Index(['Company', 'Location', 'Salary', 'Synopsis', 'Title', 'min_salary',
       'max_salary', 'avg_salary', 'analysis_yn', 'models_yn', 'software_yn',
       'sales_yn', 'security_yn', 'marketing_yn', 'experience_yn',
       'karachi_yn', 'lahore_yn', 'isl_yn', 'other_loc'],
      dtype='object')

In [94]:
X = df_os.drop('Salary', axis =1)
X = X.drop('min_salary', axis =1)
X = X.drop('max_salary', axis =1)
X = X.drop('avg_salary', axis =1)
X = X.drop('Synopsis', axis =1)
X = X.drop('Location', axis =1)
X = X.drop('Title', axis =1)
X = X.drop('Company', axis =1)
y = df_os.avg_salary
y.head()


0     35.0
1     50.0
2     50.0
3    100.0
4     85.0
Name: avg_salary, dtype: float64

In [95]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0, test_size =0.25)


# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))


23.015151515151516


In [96]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)


for max_leaf_nodes in [5,25, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))


Max leaf nodes: 5  		 Mean Absolute Error:  20
Max leaf nodes: 25  		 Mean Absolute Error:  23
Max leaf nodes: 50  		 Mean Absolute Error:  23
Max leaf nodes: 500  		 Mean Absolute Error:  23
Max leaf nodes: 5000  		 Mean Absolute Error:  23


<h4><i> Max leaf node of 5 give the least error of 20K </i></h4>

<h1> Random Forest </h1>


In [65]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
def random_forest(n_estimators,train_X,train_y,val_X,val_y):
    forest_model = RandomForestRegressor(n_estimators=n_estimators,random_state=1)
    forest_model.fit(train_X, train_y)
    melb_preds = forest_model.predict(val_X)
    print("n_estimators: %d  \t\t Mean Absolute Error:  %d" %(n_estimators, mean_absolute_error(val_y, melb_preds)))
random_forest(100,train_X,train_y,val_X,val_y)

n_estimators: 100  		 Mean Absolute Error:  19


In [66]:
for n_estimators in [5,10, 100, 500, 1000]:
    random_forest(n_estimators,train_X,train_y,val_X,val_y)

n_estimators: 5  		 Mean Absolute Error:  19
n_estimators: 10  		 Mean Absolute Error:  18
n_estimators: 100  		 Mean Absolute Error:  19
n_estimators: 500  		 Mean Absolute Error:  19
n_estimators: 1000  		 Mean Absolute Error:  19


<h4><i> n_estimators of 10 give the least error of 18K </i></h4>

<h3> Lets label encode 'Title', 'Company', 'Location' in a bigger dataset </h3>

In [125]:
#Loading all the jobs with or without salary contained in the following file 
df_ws = pd.read_csv('F:/Data_Cleaned_NoSalary.csv', index_col=0)
#Loading all the jobs having salary contained in the following file 
df_os = pd.read_csv('F:/Data_Cleaned_OnlySalary.csv', index_col=0)
# Get list of categorical variables
object_cols = ['Title','Location','Company']

#obj_cols = ['Title','Company','Location']
from sklearn.preprocessing import LabelEncoder
# Make copy to avoid changing original data 
label_X_ws = df_ws.copy()
label_X_os = df_os.copy()
for col in object_cols:
    label_X_ws[col] = label_encoder.fit_transform(df_ws[col])
    label_X_os[col] = label_encoder.transform(df_os[col])



In [128]:
X = label_X_os.drop('Salary', axis =1)
X = X.drop('Synopsis', axis =1)
X = X.drop('min_salary', axis =1)
X = X.drop('max_salary', axis =1)
X = X.drop('avg_salary', axis =1)
y = label_X_os['avg_salary']
X.head()

,Company,Location,Title,analysis_yn,models_yn,software_yn,sales_yn,security_yn,marketing_yn,experience_yn
0,25,2,32,0,0,0,1,0,0,0
1,89,2,135,1,1,0,0,0,0,0
2,89,2,134,1,1,0,0,0,0,0
3,134,2,120,0,1,0,1,0,1,0
4,31,2,145,1,1,0,0,1,0,0


In [129]:
#Decision Tree to label encoded features
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0, test_size =0.25)
for max_leaf_nodes in [5,25, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  15
Max leaf nodes: 25  		 Mean Absolute Error:  19
Max leaf nodes: 50  		 Mean Absolute Error:  19
Max leaf nodes: 500  		 Mean Absolute Error:  19
Max leaf nodes: 5000  		 Mean Absolute Error:  19


In [131]:
#Random Forest Model to label encoded features
for n_estimators in [5,10, 100, 500, 1000 ,5000]:
    random_forest(n_estimators,train_X,train_y,val_X,val_y)

n_estimators: 5  		 Mean Absolute Error:  14
n_estimators: 10  		 Mean Absolute Error:  14
n_estimators: 100  		 Mean Absolute Error:  13
n_estimators: 500  		 Mean Absolute Error:  13
n_estimators: 1000  		 Mean Absolute Error:  13
n_estimators: 5000  		 Mean Absolute Error:  13


<h4> We have narrowed MAE down to 13K by using Random forest Model to encoded features </h4>

<h2> Let's try one hot encoding to see if we can minimize MAE </h2>

In [137]:
#Loading all the jobs with or without salary contained in the following file 
df_ws = pd.read_csv('F:/Data_Cleaned_NoSalary.csv', index_col=0)
#Loading all the jobs having salary contained in the following file 
df_os = pd.read_csv('F:/Data_Cleaned_OnlySalary.csv', index_col=0)
# Get list of categorical variables
object_cols = ['Title','Location','Company']

from sklearn.preprocessing import OneHotEncoder
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_ws = pd.DataFrame(OH_encoder.fit_transform(df_ws[object_cols]))
OH_cols_os = pd.DataFrame(OH_encoder.transform(df_os[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_ws.index = df_ws.index
OH_cols_os.index = df_os.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_ws = df_ws.drop(object_cols, axis=1)
num_X_os = df_os.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_ws = pd.concat([num_X_ws, OH_cols_ws], axis=1)
OH_X_os = pd.concat([num_X_os, OH_cols_os], axis=1)

X = OH_X_os.drop('Salary', axis =1)
X = X.drop('Synopsis', axis =1)
X = X.drop('min_salary', axis =1)
X = X.drop('max_salary', axis =1)
X = X.drop('avg_salary', axis =1)
y = label_X_os['avg_salary']
X.head()


,analysis_yn,models_yn,software_yn,sales_yn,security_yn,marketing_yn,experience_yn,0,1,2,...,314,315,316,317,318,319,320,321,322,323
0,0,0,0,1,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,0,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,1,0,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0,0,1,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
#Decision Tree to label encoded features
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0, test_size =0.25)
for max_leaf_nodes in [5,25, 50, 500, 5000, 10000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  17
Max leaf nodes: 25  		 Mean Absolute Error:  15
Max leaf nodes: 50  		 Mean Absolute Error:  15
Max leaf nodes: 500  		 Mean Absolute Error:  15
Max leaf nodes: 5000  		 Mean Absolute Error:  15
Max leaf nodes: 10000  		 Mean Absolute Error:  15


In [140]:
#Random Forest Model to label encoded features
for n_estimators in [5,10, 100, 500, 1000 ,5000]:
    random_forest(n_estimators,train_X,train_y,val_X,val_y)

n_estimators: 5  		 Mean Absolute Error:  14
n_estimators: 10  		 Mean Absolute Error:  15
n_estimators: 100  		 Mean Absolute Error:  15
n_estimators: 500  		 Mean Absolute Error:  15
n_estimators: 1000  		 Mean Absolute Error:  15
n_estimators: 5000  		 Mean Absolute Error:  15


<h4> Label Encoding works better in our case </h4>

<h3> Let's assign the results (random forest model used on label encoded dataset) to the missing salary values </h3>

In [213]:
#Loading all the jobs with or without salary contained in the following file 
df_ws = pd.read_csv('F:/Data_Cleaned_NoSalary.csv', index_col=0)
#Loading all the jobs having salary contained in the following file 
df_os = pd.read_csv('F:/Data_Cleaned_OnlySalary.csv', index_col=0)
# Get list of categorical variables
object_cols = ['Title','Location','Company']
from sklearn.preprocessing import LabelEncoder
# Make copy to avoid changing original data 
label_X_ws = df_ws.copy()
label_X_os = df_os.copy()
for col in object_cols:
    label_X_ws[col] = label_encoder.fit_transform(df_ws[col])
    label_X_os[col] = label_encoder.transform(df_os[col])
X_os = label_X_os.drop('Salary', axis =1)
X_os = X_os.drop('Synopsis', axis =1)
X_os = X_os.drop('min_salary', axis =1)
X_os = X_os.drop('max_salary', axis =1)
X_os = X_os.drop('avg_salary', axis =1)
y = label_X_os['avg_salary']
forest_model = RandomForestRegressor(n_estimators=100,random_state=1)
forest_model.fit(X, y)
X_ws = label_X_ws.drop('Salary', axis =1)
X_ws = X_ws.drop('Synopsis', axis =1)
melb_preds = forest_model.predict(X_ws)
df_ws['Predicted_Salary'] = pd.DataFrame(melb_preds*1000)
df_ws['Predicted_Salary'] = df_ws['Predicted_Salary'].apply(lambda x: int(x))

In [210]:
pred_salary = df_ws.drop('analysis_yn', axis =1)
pred_salary = pred_salary.drop('models_yn', axis =1)
pred_salary = pred_salary.drop('software_yn', axis =1)
pred_salary = pred_salary.drop('sales_yn', axis =1)
pred_salary = pred_salary.drop('security_yn', axis =1)
pred_salary = pred_salary.drop('marketing_yn', axis =1)
pred_salary = pred_salary.drop('experience_yn', axis =1)
pred_salary.head(50)

,Company,Location,Salary,Synopsis,Title,Predicted_Salary
0,TOP Agency,Faisalabad,None,"If so, you might make a killer Data Scientist....",Data Scientist,38790
1,TechnoSaurs,Faisalabad,None,Work with data scientists and analysts to impr...,MERN Stack Developer,41495
2,TOP Agency,Faisalabad,None,"Our team of world-class writers, communicators...",Social Media Manager,52110
3,Cobblestone Energy,Faisalabad,None,Junior Trader / Analyst - Commercial Developme...,"JUNIOR TRADER / ANALYST - Dubai, UAE",43460
4,Sherserve,Faisalabad,None,Strong understanding of document management an...,Business Analyst,58375
5,Cobblestone Energy,Faisalabad,None,Junior Trader / Analyst - Commercial Developme...,"JUNIOR TRADER / ANALYST - Dubai, UAE",43460
6,Sherserve,Faisalabad,None,Involvement in requirements analysis in coordi...,Senior Software Developer,49130
7,CHALO,Karachi,"Rs 35,000 a month",Online exam set up and data entry.Working in o...,Customer Support Representative,40525
8,Medusa Pakistan,None,None,Performs data ports and data loads;Monitors an...,Database Administrator,29950
9,Cobblestone Energy,Karachi,None,High level of interest in data;For the UK mark...,"CHIEF TECHNOLOGY OFFICER - Dubai , UAE",47425
